<span style="font-size:2em;">Introduction / Business Problem</span>
<p><u><i>Goal</i></u></p>
<p>To identify the ideal location to open a restaurant in Singapore.</p>
<p><u><i>Methodology</i></u></p>
<p>Utilising data from multiple sources to identify clusters of restaurants in Singapore.</p>
<p><a href="https://en.wikipedia.org/wiki/Postal_codes_in_Singapore"> Wikipedia</a>: Singapore postal codes</p>
<p><a href="https://www.sglocate.com/"> SGLocate</a>: API to obtain latitude and longtitude for each Singapore postal codes</p>
<p><a href="https://foursquare.com/discoversing/"> SGLocate</a>: Use foursquare to identify clusters and determine ideal location to open a restaurant</p>

<span style="font-size:2em;">Obtain zipcodes from wikipedia</span>

In [1]:
import pandas as pd
import numpy as np
from pandas.io.html import read_html

#Read wikitables in the wikipedia page
page = 'https://en.wikipedia.org/wiki/Postal_codes_in_Singapore'
wikitable = pd.read_html(page, index_col = 0, attrs={"class":"wikitable"})

#write table into dataframe
sg_post = wikitable[0]
sg_post.reset_index(inplace=True)

sg_post2 = sg_post.rename(columns={'Postal district': 'postal_district', 'Postal sector(1st 2 digits of 6-digit postal codes)': 'postal_sector', 'General location': 'general_location'})
sg_post2['seq'] = sg_post2.index

postal_sector = sg_post2[{'seq', 'postal_sector'}]
postal_location = sg_post2[{'seq','general_location'}]
postal_district = sg_post2[{'seq','postal_district'}]

sg_post2.head()


,postal_district,postal_sector,general_location,seq
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park",0
1,2,"07, 08","Anson, Tanjong Pagar",1
2,3,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru",2
3,4,"09, 10","Telok Blangah, Harbourfront",3
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New...",4


As you can see, multiple postal sector are grouped into 1 row, we will have to split them

In [2]:
def pir(df, c):
    colc = df[c].str.split(',')
    clst = colc.values.tolist()
    lens = [len(l) for l in clst]

    cdf = pd.DataFrame({c: np.concatenate(clst)}, df.index.repeat(lens))
    return df.drop(c, 1).join(cdf).reset_index(drop=True)

postal_sector2 = pir(postal_sector, 'postal_sector')
#postal_location2 = pir(postal_location, 'general_location')
postal_sector2.head()

,seq,postal_sector
0,0,01
1,0,02
2,0,03
3,0,04
4,0,05


Convert postal district from interget to string in order to calculate length

In [3]:
postal_sector2['postal_sector'] = postal_sector2['postal_sector'].str.strip()
postal_district['postal_district'] = postal_district['postal_district'].astype(str)
postal_district.dtypes

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


postal_district    object
seq                 int64
dtype: object

Merge district, sector and location back into 1 dataframe and move the key 'seq' back to the front of the dataframe

In [4]:
final_post = postal_district.merge(postal_sector2, on= 'seq', how='left', sort = True)
final_post = final_post.merge(postal_location, how='left', on='seq', sort = True)

seq = final_post['seq']
final_post.drop(labels=['seq'], axis=1,inplace = True)
final_post.insert(0, 'seq', seq)
final_post.head()

,seq,postal_district,postal_sector,general_location
0,0,1,01,"Raffles Place, Cecil, Marina, People's Park"
1,0,1,02,"Raffles Place, Cecil, Marina, People's Park"
2,0,1,03,"Raffles Place, Cecil, Marina, People's Park"
3,0,1,04,"Raffles Place, Cecil, Marina, People's Park"
4,0,1,05,"Raffles Place, Cecil, Marina, People's Park"


Singapore postal codes consists of 6 digit. Depending on the number of digit in postal district, the number of leading zeros that needs to be added would be different. Calculate the length of both postal sector and district

In [5]:
final_post['sector_len'] = final_post['postal_sector'].str.len()
final_post['postal_len'] = final_post['postal_district'].str.len()
final_post.head()

,seq,postal_district,postal_sector,general_location,sector_len,postal_len
0,0,1,01,"Raffles Place, Cecil, Marina, People's Park",2,1
1,0,1,02,"Raffles Place, Cecil, Marina, People's Park",2,1
2,0,1,03,"Raffles Place, Cecil, Marina, People's Park",2,1
3,0,1,04,"Raffles Place, Cecil, Marina, People's Park",2,1
4,0,1,05,"Raffles Place, Cecil, Marina, People's Park",2,1


Fill postal sector with the appropriate leading zeros based on postal district length

In [6]:
final_post['new_sector'] = np.where(final_post['postal_len']==1, final_post['postal_sector'].str.zfill(5),final_post['postal_sector'].str.zfill(4))

final_post.head()

,seq,postal_district,postal_sector,general_location,sector_len,postal_len,new_sector
0,0,1,01,"Raffles Place, Cecil, Marina, People's Park",2,1,00001
1,0,1,02,"Raffles Place, Cecil, Marina, People's Park",2,1,00002
2,0,1,03,"Raffles Place, Cecil, Marina, People's Park",2,1,00003
3,0,1,04,"Raffles Place, Cecil, Marina, People's Park",2,1,00004
4,0,1,05,"Raffles Place, Cecil, Marina, People's Park",2,1,00005


Join the postal district with the newly added sector with leading zeros to derive the final Singapore Zip Codes

In [7]:
final_post['final_zipcode'] = final_post['postal_district'] + final_post['new_sector']

final_post.drop(['sector_len', 'postal_len', 'new_sector'],axis=1 , inplace=True)
final_post.head()

,seq,postal_district,postal_sector,general_location,final_zipcode
0,0,1,01,"Raffles Place, Cecil, Marina, People's Park",100001
1,0,1,02,"Raffles Place, Cecil, Marina, People's Park",100002
2,0,1,03,"Raffles Place, Cecil, Marina, People's Park",100003
3,0,1,04,"Raffles Place, Cecil, Marina, People's Park",100004
4,0,1,05,"Raffles Place, Cecil, Marina, People's Park",100005
